Packages import

In [1]:
from data import get_dataset
from model import features,classifier,metrics
import tqdm

GPU

In [6]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

UsageError: Line magic function `%tensorflow_version` not found.


Data processing

In [2]:
path="random_split/"

test,val,train =get_dataset.get_df(path)

#Due to limitation of computing capacities, we will only consider the 1000 most important classes
classes=features.get_classes_top1000(train)
#apply those changes to train, test, val
test=features.reduce_dataset_1000topclasses(test,classes)
train=features.reduce_dataset_1000topclasses(train,classes)
val=features.reduce_dataset_1000topclasses(val,classes)

#Process inputs 
train_processed=features.process_dataset(train, classes)
test_processed=features.process_dataset(test, classes)
val_processed=features.process_dataset(val, classes)
print(train_processed.shape)
print(test_processed.shape)
print(val_processed.shape)
#Process labels
y_train=features.process_labels(train,classes)
y_test=features.process_labels(test,classes)
y_val=features.process_labels(val,classes)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(439493, 100)
(54378, 100)
(54378, 100)
(439493, 1000)
(54378, 1000)
(54378, 1000)


Simple model : using LSTM

In [3]:
from model import features,classifier,metrics
import tqdm
model=classifier.build_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          2100      
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1000)              129000    
Total params: 215,580
Trainable params: 215,580
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
history=classifier.train_model(model,train_processed,y_train,val_processed,y_val,epochs=2)
classifier.plot_accuracy_train_val(history)

Epoch 1/2
1717/1717 [==============================] - 1726s 1s/step - loss: 6.9618 - accuracy: 0.0426 - val_loss: 3.7486 - val_accuracy: 0.3070
Epoch 2/2
1717/1717 [==============================] - 1682s 980ms/step - loss: 3.3797 - accuracy: 0.3645 - val_loss: 2.1645 - val_accuracy: 0.6343


NameError: name 'model_lstm_wieghts' is not defined

ProtCNN